# 15-1_test(FTtransformer) 요약

- 모델: MLP + FT-Transformer 앙상블(예측 평균)
- 피처: Q_A/Q_E 로그, WR/WF, TP Big5, 인구통계/플래그, interaction + Target Encoding
- 학습/평가: StratifiedKFold 5, seed=42, ROC-AUC 기준
- 제출파일: submission_blend.csv


In [ ]:
import os
import random
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import roc_auc_score

# ============================================================
# CONFIG
# ============================================================
SEED = 42
N_FOLDS = 5
BATCH_SIZE = 512
EPOCHS = 50
PATIENCE = 8
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

print(f"🖥️ Device: {DEVICE}")


def set_seed(seed=SEED):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)


set_seed(SEED)


# ============================================================
# 데이터 로딩
# ============================================================
print("\n📂 데이터 로딩...")
train_raw = pd.read_csv("../../data/raw/train.csv")
test_raw = pd.read_csv("../../data/raw/test_x.csv")

train_raw["voted_bin"] = (train_raw["voted"] == 2).astype(int)
print(f"Train: {train_raw.shape}, Test: {test_raw.shape}")


# ============================================================
# 이상치/무응답 처리
# ============================================================
def clean_data(df):
    df = df.copy()
    
    # 무응답 (0 → NaN)
    for col in ['education', 'engnat', 'hand', 'married', 'urban']:
        if col in df.columns:
            df.loc[df[col] == 0, col] = np.nan
    
    # familysize
    if 'familysize' in df.columns:
        df.loc[df['familysize'] == 0, 'familysize'] = np.nan
        df.loc[df['familysize'] > 15, 'familysize'] = np.nan
    
    # TP 0 → NaN
    for col in [f"tp{i:02d}" for i in range(1, 11)]:
        if col in df.columns:
            df.loc[df[col] == 0, col] = np.nan
    
    # Q_E 클리핑
    for col in [f"Q{c}E" for c in "abcdefghijklmnopqrst"]:
        if col in df.columns:
            df[col] = df[col].clip(lower=100, upper=60000)
    
    return df


# ============================================================
# 피처 엔지니어링
# ============================================================
def build_features(df):
    df = df.copy()
    
    # 인구통계
    age_map = {"10s": 1, "20s": 2, "30s": 3, "40s": 4, "50s": 5, "60s": 6, "+70s": 7}
    df["age_ord"] = df["age_group"].map(age_map)
    df["is_teenager"] = (df["age_ord"] == 1).astype(int)
    df["is_young"] = (df["age_ord"] <= 2).astype(int)
    df["is_old"] = (df["age_ord"] >= 6).astype(int)
    df["edu_low"] = (df["education"] <= 2).astype(float)
    df["edu_high"] = (df["education"] >= 3).astype(float)
    df["is_single"] = (df["married"] == 1).astype(float)
    df["is_married"] = (df["married"] == 2).astype(float)
    df["is_urban"] = (df["urban"] == 3).astype(float)
    df["is_english_native"] = (df["engnat"] == 1).astype(float)
    df["is_male"] = (df["gender"] == "Male").astype(int)
    
    # Q_A 집계
    qa_cols = [f"Q{c}A" for c in "abcdefghijklmnopqrst"]
    df["qa_mean"] = df[qa_cols].mean(axis=1)
    df["qa_std"] = df[qa_cols].std(axis=1)
    df["qa_range"] = df[qa_cols].max(axis=1) - df[qa_cols].min(axis=1)
    df["qa_extreme_ratio"] = ((df[qa_cols] == 1) | (df[qa_cols] == 5)).sum(axis=1) / 20
    df["qa_neutral_ratio"] = (df[qa_cols] == 3).sum(axis=1) / 20
    df["qa_all_same"] = (df[qa_cols].std(axis=1) == 0).astype(int)
    
    # Q_E 로그 + 집계
    qe_cols = [f"Q{c}E" for c in "abcdefghijklmnopqrst"]
    for col in qe_cols:
        df[f"{col}_log"] = np.log1p(df[col])
    
    qe_log_cols = [f"{col}_log" for col in qe_cols]
    df["qe_log_mean"] = df[qe_log_cols].mean(axis=1)
    df["qe_log_std"] = df[qe_log_cols].std(axis=1)
    df["qe_fast_ratio"] = (df[qe_cols] < 500).sum(axis=1) / 20
    df["qe_total_log"] = df[qe_log_cols].sum(axis=1)
    df["is_careless"] = ((df[qe_cols].mean(axis=1) < 500) | (df["qa_all_same"] == 1)).astype(int)
    
    # TP Big5
    tp_cols = [f"tp{i:02d}" for i in range(1, 11)]
    df["tp_missing_ratio"] = df[tp_cols].isna().sum(axis=1) / 10
    df["extraversion"] = df["tp01"] - df["tp06"]
    df["agreeableness"] = df["tp07"] - df["tp02"]
    df["conscientiousness"] = df["tp03"] - df["tp08"]
    df["neuroticism"] = df["tp04"] - df["tp09"]
    df["openness"] = df["tp05"] - df["tp10"]
    df["tp_mean"] = df[tp_cols].mean(axis=1)
    
    # WR/WF
    wr_cols = [f"wr_{i:02d}" for i in range(1, 14)]
    wf_cols = [f"wf_{i:02d}" for i in range(1, 4)]
    df["wr_sum"] = df[wr_cols].sum(axis=1)
    df["wf_sum"] = df[wf_cols].sum(axis=1)
    df["word_credibility"] = df["wr_sum"] - df["wf_sum"]
    df["vocab_high"] = (df["wr_sum"] >= 11).astype(int)
    
    # Interaction
    df["age_edu"] = df["age_ord"] * df["education"]
    df["young_low_edu"] = df["is_young"] * df["edu_low"]
    df["young_single"] = df["is_young"] * df["is_single"]
    df["old_married"] = df["is_old"] * df["is_married"]
    df["teenager_low_edu"] = df["is_teenager"] * df["edu_low"]
    
    return df


# ============================================================
# Target Encoding
# ============================================================
def target_encode(train_df, val_df, test_df, col, target_col, smoothing=10):
    global_mean = train_df[target_col].mean()
    agg = train_df.groupby(col)[target_col].agg(['mean', 'count'])
    agg['te'] = (agg['count'] * agg['mean'] + smoothing * global_mean) / (agg['count'] + smoothing)
    te_map = agg['te'].to_dict()
    
    return (train_df[col].map(te_map).fillna(global_mean).values,
            val_df[col].map(te_map).fillna(global_mean).values,
            test_df[col].map(te_map).fillna(global_mean).values)


def create_target_encodings(train_df, val_df, test_df, target_col="voted_bin"):
    te_dict = {'train': {}, 'val': {}, 'test': {}}
    
    # 단일
    for col in ['age_group', 'race', 'religion']:
        tr, va, te = target_encode(train_df, val_df, test_df, col, target_col, 10)
        te_dict['train'][f'{col}_te'] = tr
        te_dict['val'][f'{col}_te'] = va
        te_dict['test'][f'{col}_te'] = te
    
    # 복합
    for df in [train_df, val_df, test_df]:
        df['age_edu_cat'] = df['age_group'].astype(str) + '_' + df['education'].astype(str)
        df['age_married_cat'] = df['age_group'].astype(str) + '_' + df['married'].astype(str)
        df['age_race_cat'] = df['age_group'].astype(str) + '_' + df['race'].astype(str)
        df['age_edu_married_cat'] = df['age_group'].astype(str) + '_' + df['education'].astype(str) + '_' + df['married'].astype(str)
    
    for col, sm in [('age_edu_cat', 5), ('age_married_cat', 5), ('age_race_cat', 5), ('age_edu_married_cat', 3)]:
        tr, va, te = target_encode(train_df, val_df, test_df, col, target_col, sm)
        te_dict['train'][f'{col}_te'] = tr
        te_dict['val'][f'{col}_te'] = va
        te_dict['test'][f'{col}_te'] = te
    
    return te_dict


# ============================================================
# 피처 정의
# ============================================================
qa_cols = [f"Q{c}A" for c in "abcdefghijklmnopqrst"]
qe_log_cols = [f"Q{c}E_log" for c in "abcdefghijklmnopqrst"]
wr_cols = [f"wr_{i:02d}" for i in range(1, 14)]
wf_cols = [f"wf_{i:02d}" for i in range(1, 4)]
tp_cols = [f"tp{i:02d}" for i in range(1, 11)]

num_features = (
    qa_cols + qe_log_cols + wr_cols + wf_cols + tp_cols +
    [
        "age_ord", "education", "married", "urban", "engnat", "familysize", "hand",
        "is_teenager", "is_young", "is_old", "edu_low", "edu_high",
        "is_single", "is_married", "is_urban", "is_english_native", "is_male",
        "qa_mean", "qa_std", "qa_range", "qa_extreme_ratio", "qa_neutral_ratio", "qa_all_same",
        "qe_log_mean", "qe_log_std", "qe_fast_ratio", "qe_total_log", "is_careless",
        "tp_missing_ratio", "tp_mean",
        "extraversion", "agreeableness", "conscientiousness", "neuroticism", "openness",
        "wr_sum", "wf_sum", "word_credibility", "vocab_high",
        "age_edu", "young_low_edu", "young_single", "old_married", "teenager_low_edu",
    ]
)

te_features = ['age_group_te', 'race_te', 'religion_te', 
               'age_edu_cat_te', 'age_married_cat_te', 'age_race_cat_te', 'age_edu_married_cat_te']

cat_features = ['gender', 'race', 'religion']

print(f"📊 피처: num={len(num_features)}, te={len(te_features)}, cat={len(cat_features)}")


# ============================================================
# Dataset
# ============================================================
class TabDataset(Dataset):
    def __init__(self, X_num, X_cat, y=None):
        self.X_num = torch.tensor(X_num, dtype=torch.float32)
        self.X_cat = torch.tensor(X_cat, dtype=torch.long)
        self.y = None if y is None else torch.tensor(y, dtype=torch.float32).unsqueeze(1)
    
    def __len__(self):
        return len(self.X_num)
    
    def __getitem__(self, idx):
        if self.y is None:
            return self.X_num[idx], self.X_cat[idx]
        return self.X_num[idx], self.X_cat[idx], self.y[idx]


# ============================================================
# Model 1: MLP
# ============================================================
class MLP(nn.Module):
    def __init__(self, num_features, cat_dims, embed_dim=8, hidden_dims=[256, 128, 64], dropout=0.3):
        super().__init__()
        self.embeddings = nn.ModuleList([nn.Embedding(dim + 1, embed_dim) for dim in cat_dims])
        
        input_dim = num_features + len(cat_dims) * embed_dim
        layers = []
        prev_dim = input_dim
        
        for hidden_dim in hidden_dims:
            layers.extend([
                nn.Linear(prev_dim, hidden_dim),
                nn.BatchNorm1d(hidden_dim),
                nn.ReLU(),
                nn.Dropout(dropout),
            ])
            prev_dim = hidden_dim
        
        self.mlp = nn.Sequential(*layers)
        self.output = nn.Linear(hidden_dims[-1], 1)
        self._init_weights()
    
    def _init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    nn.init.zeros_(m.bias)
    
    def forward(self, x_num, x_cat):
        cat_embeds = torch.cat([emb(x_cat[:, i]) for i, emb in enumerate(self.embeddings)], dim=1)
        x = torch.cat([x_num, cat_embeds], dim=1)
        x = self.mlp(x)
        return self.output(x)


# ============================================================
# Model 2: FT-Transformer (경량화)
# ============================================================
class NumericalEmbedding(nn.Module):
    def __init__(self, num_features, d_token):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(num_features, d_token) * 0.02)
        self.bias = nn.Parameter(torch.zeros(num_features, d_token))
    
    def forward(self, x):
        return x.unsqueeze(-1) * self.weight + self.bias


class FTTransformer(nn.Module):
    def __init__(self, num_features, cat_dims, d_token=48, n_layers=2, n_heads=4, dropout=0.2):
        super().__init__()
        self.num_embed = NumericalEmbedding(num_features, d_token)
        self.cat_embeds = nn.ModuleList([nn.Embedding(dim + 1, d_token) for dim in cat_dims])
        self.cls_token = nn.Parameter(torch.randn(1, 1, d_token) * 0.02)
        
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_token, nhead=n_heads, dim_feedforward=d_token * 2,
            dropout=dropout, activation='gelu', batch_first=True, norm_first=True
        )
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=n_layers)
        
        self.head = nn.Sequential(
            nn.LayerNorm(d_token),
            nn.Linear(d_token, d_token // 2),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(d_token // 2, 1)
        )
    
    def forward(self, x_num, x_cat):
        num_tokens = self.num_embed(x_num)
        cat_tokens = torch.stack([emb(x_cat[:, i]) for i, emb in enumerate(self.cat_embeds)], dim=1)
        
        tokens = torch.cat([num_tokens, cat_tokens], dim=1)
        cls = self.cls_token.expand(tokens.size(0), -1, -1)
        tokens = torch.cat([cls, tokens], dim=1)
        
        x = self.transformer(tokens)
        return self.head(x[:, 0])


# ============================================================
# 학습 함수
# ============================================================
def train_model(model, train_loader, val_loader, val_y, device, epochs=EPOCHS, patience=PATIENCE, lr=1e-3):
    model.to(device)
    
    pos_ratio = val_y.mean()
    pos_weight = torch.tensor([(1 - pos_ratio) / (pos_ratio + 1e-6)], device=device)
    criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
    
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=1e-4)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=3)
    
    best_auc = 0
    best_state = None
    no_improve = 0
    
    for epoch in range(epochs):
        model.train()
        for X_num, X_cat, y in train_loader:
            X_num, X_cat, y = X_num.to(device), X_cat.to(device), y.to(device)
            optimizer.zero_grad()
            loss = criterion(model(X_num, X_cat), y)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
        
        model.eval()
        val_preds = []
        with torch.no_grad():
            for X_num, X_cat, _ in val_loader:
                X_num, X_cat = X_num.to(device), X_cat.to(device)
                val_preds.append(torch.sigmoid(model(X_num, X_cat)).cpu().numpy())
        
        val_preds = np.concatenate(val_preds).ravel()
        val_auc = roc_auc_score(val_y, val_preds)
        scheduler.step(val_auc)
        
        if val_auc > best_auc + 1e-5:
            best_auc = val_auc
            best_state = {k: v.cpu().clone() for k, v in model.state_dict().items()}
            no_improve = 0
        else:
            no_improve += 1
        
        if no_improve >= patience:
            break
    
    if best_state:
        model.load_state_dict(best_state)
    return model, best_auc


def predict(model, loader, device):
    model.eval()
    preds = []
    with torch.no_grad():
        for batch in loader:
            X_num, X_cat = batch[0].to(device), batch[1].to(device)
            preds.append(torch.sigmoid(model(X_num, X_cat)).cpu().numpy())
    return np.concatenate(preds).ravel()


# ============================================================
# 메인
# ============================================================
def main():
    print("\n" + "="*60)
    print("🚀 MLP + FT-Transformer 앙상블 학습")
    print("="*60)
    
    set_seed(SEED)
    
    train_clean = clean_data(train_raw)
    test_clean = clean_data(test_raw)
    
    # 결과 저장
    oof_mlp = np.zeros(len(train_clean))
    oof_ft = np.zeros(len(train_clean))
    test_mlp = np.zeros(len(test_clean))
    test_ft = np.zeros(len(test_clean))
    
    fold_aucs_mlp = []
    fold_aucs_ft = []
    
    skf = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=SEED)
    
    for fold, (train_idx, val_idx) in enumerate(skf.split(train_clean, train_clean["voted_bin"])):
        print(f"\n{'='*50}")
        print(f"📂 Fold {fold + 1}/{N_FOLDS}")
        print(f"{'='*50}")
        
        train_fold = train_clean.iloc[train_idx].copy().reset_index(drop=True)
        val_fold = train_clean.iloc[val_idx].copy().reset_index(drop=True)
        test_fold = test_clean.copy()
        
        # 피처 엔지니어링
        train_fe = build_features(train_fold)
        val_fe = build_features(val_fold)
        test_fe = build_features(test_fold)
        
        # Target Encoding
        te_dict = create_target_encodings(train_fe, val_fe, test_fe, "voted_bin")
        
        # 수치형 피처
        all_num = num_features + te_features
        
        X_train = train_fe[num_features].copy()
        X_val = val_fe[num_features].copy()
        X_test = test_fe[num_features].copy()
        
        for te_name in te_features:
            X_train[te_name] = te_dict['train'][te_name]
            X_val[te_name] = te_dict['val'][te_name]
            X_test[te_name] = te_dict['test'][te_name]
        
        # NaN → 중앙값
        for col in all_num:
            med = X_train[col].median()
            if pd.isna(med):
                med = 0
            X_train[col] = X_train[col].fillna(med)
            X_val[col] = X_val[col].fillna(med)
            X_test[col] = X_test[col].fillna(med)
        
        # 스케일링
        scaler = StandardScaler()
        X_train_s = scaler.fit_transform(X_train.values)
        X_val_s = scaler.transform(X_val.values)
        X_test_s = scaler.transform(X_test.values)
        
        # 범주형
        cat_dims = []
        X_cat_train, X_cat_val, X_cat_test = [], [], []
        
        for col in cat_features:
            le = LabelEncoder()
            all_vals = list(set(train_fe[col].fillna("NaN").astype(str)) | 
                           set(val_fe[col].fillna("NaN").astype(str)) | 
                           set(test_fe[col].fillna("NaN").astype(str)))
            le.fit(all_vals + ["UNK"])
            cat_dims.append(len(le.classes_))
            
            X_cat_train.append(le.transform(train_fe[col].fillna("NaN").astype(str)))
            X_cat_val.append(le.transform(val_fe[col].fillna("NaN").astype(str)))
            X_cat_test.append(le.transform(test_fe[col].fillna("NaN").astype(str)))
        
        X_cat_train = np.stack(X_cat_train, axis=1)
        X_cat_val = np.stack(X_cat_val, axis=1)
        X_cat_test = np.stack(X_cat_test, axis=1)
        
        y_train = train_fe["voted_bin"].values.astype(np.float32)
        y_val = val_fe["voted_bin"].values.astype(np.float32)
        
        # DataLoader
        train_ds = TabDataset(X_train_s, X_cat_train, y_train)
        val_ds = TabDataset(X_val_s, X_cat_val, y_val)
        test_ds = TabDataset(X_test_s, X_cat_test)
        
        train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
        val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False)
        test_loader = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False)
        
        # =====================
        # Model 1: MLP
        # =====================
        print("\n  🔷 MLP 학습...")
        mlp = MLP(num_features=len(all_num), cat_dims=cat_dims, embed_dim=8, 
                  hidden_dims=[256, 128, 64], dropout=0.3)
        mlp, auc_mlp = train_model(mlp, train_loader, val_loader, y_val, DEVICE, lr=1e-3)
        fold_aucs_mlp.append(auc_mlp)
        print(f"    MLP AUC: {auc_mlp:.5f}")
        
        oof_mlp[val_idx] = predict(mlp, val_loader, DEVICE)
        test_mlp += predict(mlp, test_loader, DEVICE) / N_FOLDS
        
        # =====================
        # Model 2: FT-Transformer
        # =====================
        print("\n  🔶 FT-Transformer 학습...")
        ft = FTTransformer(num_features=len(all_num), cat_dims=cat_dims,
                          d_token=48, n_layers=2, n_heads=4, dropout=0.2)
        ft, auc_ft = train_model(ft, train_loader, val_loader, y_val, DEVICE, lr=5e-4)
        fold_aucs_ft.append(auc_ft)
        print(f"    FT AUC: {auc_ft:.5f}")
        
        oof_ft[val_idx] = predict(ft, val_loader, DEVICE)
        test_ft += predict(ft, test_loader, DEVICE) / N_FOLDS
    
    # =====================
    # 앙상블
    # =====================
    print("\n" + "="*60)
    print("🎯 앙상블 결과")
    print("="*60)
    
    # 개별 모델 OOF
    oof_auc_mlp = roc_auc_score(train_clean["voted_bin"], oof_mlp)
    oof_auc_ft = roc_auc_score(train_clean["voted_bin"], oof_ft)
    
    print(f"\n📊 MLP:")
    print(f"   OOF AUC: {oof_auc_mlp:.5f}")
    print(f"   Fold AUCs: {[f'{x:.5f}' for x in fold_aucs_mlp]}")
    
    print(f"\n📊 FT-Transformer:")
    print(f"   OOF AUC: {oof_auc_ft:.5f}")
    print(f"   Fold AUCs: {[f'{x:.5f}' for x in fold_aucs_ft]}")
    
    # 앙상블 (평균)
    oof_ensemble = (oof_mlp + oof_ft) / 2
    test_ensemble = (test_mlp + test_ft) / 2
    oof_auc_ensemble = roc_auc_score(train_clean["voted_bin"], oof_ensemble)
    
    print(f"\n🏆 앙상블 (MLP + FT 평균):")
    print(f"   OOF AUC: {oof_auc_ensemble:.5f}")
    
    # 가중 앙상블 시도
    best_w = 0.5
    best_auc = oof_auc_ensemble
    
    for w in [0.3, 0.4, 0.5, 0.6, 0.7]:
        oof_w = w * oof_mlp + (1 - w) * oof_ft
        auc_w = roc_auc_score(train_clean["voted_bin"], oof_w)
        if auc_w > best_auc:
            best_auc = auc_w
            best_w = w
    
    print(f"\n🏆 최적 가중 앙상블 (MLP:{best_w:.1f} + FT:{1-best_w:.1f}):")
    print(f"   OOF AUC: {best_auc:.5f}")
    
    # 최종 예측
    final_test = best_w * test_mlp + (1 - best_w) * test_ft
    
    # 저장
    sub = pd.DataFrame({
        "index": test_raw["index"] if "index" in test_raw.columns else range(len(test_raw)),
        "voted": final_test
    })
    sub.to_csv("submission_ensemble.csv", index=False)
    print(f"\n💾 저장: submission_ensemble(15).csv")
    print(f"   예측 범위: [{final_test.min():.4f}, {final_test.max():.4f}]")
    
    # 개별 모델도 저장
    sub_mlp = sub.copy()
    sub_mlp["voted"] = test_mlp
    sub_mlp.to_csv("submission_mlp.csv", index=False)
    
    sub_ft = sub.copy()
    sub_ft["voted"] = test_ft
    sub_ft.to_csv("submission_ft.csv", index=False)
    
    print(f"   (개별 저장: submission_mlp(15).csv, submission_ft.csv(15))")
    
    return best_auc


if __name__ == "__main__":
    main()

🖥️ Device: cpu

📂 데이터 로딩...
Train: (45532, 79), Test: (11383, 77)
📊 피처: num=110, te=7, cat=3

🚀 MLP + FT-Transformer 앙상블 학습

📂 Fold 1/5

  🔷 MLP 학습...
    MLP AUC: 0.78113

  🔶 FT-Transformer 학습...
    FT AUC: 0.78151

📂 Fold 2/5

  🔷 MLP 학습...
    MLP AUC: 0.77052

  🔶 FT-Transformer 학습...
    FT AUC: 0.76834

📂 Fold 3/5

  🔷 MLP 학습...
    MLP AUC: 0.76419

  🔶 FT-Transformer 학습...
    FT AUC: 0.75922

📂 Fold 4/5

  🔷 MLP 학습...
    MLP AUC: 0.76518

  🔶 FT-Transformer 학습...
    FT AUC: 0.76286

📂 Fold 5/5

  🔷 MLP 학습...
    MLP AUC: 0.77083

  🔶 FT-Transformer 학습...
    FT AUC: 0.76567

🎯 앙상블 결과

📊 MLP:
   OOF AUC: 0.77016
   Fold AUCs: ['0.78113', '0.77052', '0.76419', '0.76518', '0.77083']

📊 FT-Transformer:
   OOF AUC: 0.76219
   Fold AUCs: ['0.78151', '0.76834', '0.75922', '0.76286', '0.76567']

🏆 앙상블 (MLP + FT 평균):
   OOF AUC: 0.76955

🏆 최적 가중 앙상블 (MLP:0.7 + FT:0.3):
   OOF AUC: 0.77078

💾 저장: submission_ensemble(15).csv
   예측 범위: [0.1067, 0.9954]
   (개별 저장: submission_mlp(15).cs

In [2]:
import pandas as pd

v4 = pd.read_csv("submission_15_new_5th_test.csv")
v5 = pd.read_csv("submission_ensemble.csv")

blend = v4.copy()
blend["voted"] = (v4["voted"] + v5["voted"]) / 2

blend.to_csv("submission_blend.csv", index=False)